<a href="https://colab.research.google.com/github/18K41A0479/NNDL-18K41A0479/blob/main/Assi6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

**Read Data**

In [2]:
data=pd.read_csv("kc_house_data.csv")
data.head()

FileNotFoundError: ignored

In [ ]:
data.info()

**Outliers**

In [ ]:
z=np.abs(stats.zscore(data['price']))
print(z)

In [ ]:
sns.boxplot(x=data['price'])

In [ ]:
df=pd.DataFrame(data=data)
df.head()

In [ ]:
x=df
df.drop(['date'],axis=1)
for i in df:
  if i!='date':
    q=df[i].quantile(0.99)
    df[df[i]<q]
    q_low=df[i].quantile(0.01)
    q_hi=df[i].quantile(0.99)
    df_filtered=df[(df[i]<q_hi)&(df[i]>q_low)]
df_filtered.head()

In [ ]:
print(len(df))
print(len(df_filtered))
if(len(df)==len(df_filtered)):
  print('No outliers')
data=df_filtered

**Normalisation**

In [ ]:
data_n=data.copy()
for i in data:
  if i!='date':
    data_n[i]=data[i].copy()
    data_n[i]=(data_n[i]-data_n[i].min())/(data_n[i].max()-data_n[i].min())
data_n.head()

**Split Data**

In [ ]:
train_data,test_data=train_test_split(data_n,test_size=0.1)

In [ ]:
print(len(train_data))
train_data.head()

In [ ]:
x=np.array(train_data['sqft_living'])
y=np.array(train_data['price'])

**Train Data**

In [ ]:
#step 1 initialise values
bs=2
m=1.0 
c=-1.0  
l=0.1      #Learning rate
epochs=100 #Maximum iterations
ns=len(x)
nb=ns//bs
ms=ns%bs
err=[]
k=nb
if(ms>0):
  er=0
  g_m=0
  g_c=0
  for j in range(0,ms):
    er+=((y[j]-(m*x[j])-c))**2
    ind=k*bs + j
    g_m=g_m+((-1.0)*((y[ind]-m*x[ind]-c)*x[ind]))
    g_c=g_c+((-1.0)*(y[ind]-m*x[ind]-c))
  g_m=g_m/ms
  g_c=g_c/ms
  #find change in m , c
  dm=(-1.0)*l*g_m
  dc=(-1.0)*l*g_c
  #update variable 
  m=m+dm 
  c=c+dc
  er=(1/(2*len(train_data['sqft_living'])))*(er)
  err.append(er)
else:
  for i in range(epochs):
    for k in range(nb):
      er=0
      g_m=0
      g_c=0
      for j in range(bs):
        er+=((y[j]-(m*x[j])-c))**2
        ind=k*bs + j
        g_m=g_m+((-1.0)*((y[ind]-m*x[ind]-c)*x[ind]))
        g_c=g_c+((-1.0)*(y[ind]-m*x[ind]-c))
      g_m=g_m/bs
      g_c=g_c/bs
      #find change in m,c
      dm=(-1.0)*l*g_m
      dc=(-1.0)*l*g_c
      #update variable 
      m=m+dm 
      c=c+dc
      er=(1/(2*len(train_data['sqft_living'])))*(er)
      err.append(er)
#step7 print variable x1, y1
m=round(m,2)
c=round(c,2)
print("Minimum value obtained at m = {} ,c ={} for given function ".format(m,c))

**RMSE for Training data:**

In [ ]:
yp=[]
for i in range(len(x)):
  p=m*x[i]+c
  yp.append(p)
sum=0
for i in range(len(x)):
  sum+=(y[i]-yp[i])**2
  mse=sum/len(x)
print("Mean Square Error (MSE) : ",mse)

In [ ]:
plt.plot(np.array(train_data['sqft_living']),np.array(yp),'green')
plt.scatter(np.array(train_data['sqft_living']),np.array(train_data['price']),color='red')
plt.title("Predicted data vs Actual Data")

In [ ]:
iters=np.arange(epochs*nb)
plt.plot(iters,err,'green')
plt.title("Error graph for training data")
plt.xlabel("iteration count")
plt.ylabel("error")
plt.grid()

**Test Data:**

In [ ]:
tx=np.array(test_data['sqft_living'])
ty=np.array(test_data['price'])

yp1=[]
for i in range(len(tx)):
  p1=m*tx[i]+c
  yp1.append(p1)
sum=0
for i in range(len(tx)):
  sum+=(ty[i]-yp1[i])**2
  mse1=sum/len(tx)
print("Mean Square Error (MSE) : ",mse1)

In [ ]:
plt.plot(test_data['price'],yp1,'green')
plt.grid()

In [ ]:
x_min=data['sqft_living'].min()
x_max=data['sqft_living'].max()
y_min=data['price'].min()
y_max=data['price'].max()

**Predict the Output:**

In [ ]:
x2=float(input("Enter sqrt feet values sqft_living , sqft_lot, sqft_above, sqft_basement, sqft_living15, sqft_lot15 value : "))
x2=(x2-x_min)/(x_max-x_min)
y2=m*x2+c
y2=y2*(y_max-y_min)+y_min
print("Price  is : {}".format(y2))